In [27]:
import tensorflow as tf
import numpy as np
import gym
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from collections import deque

In [18]:
env = gym.make('FrozenLake-v0')
print(env.action_space.n)
print(env.observation_space.n)

[2017-07-05 23:55:37,586] Making new env: FrozenLake-v0


4
16


In [19]:
env.reset()
for i in range(500):
    # env.render()
    state, reward, done, prob = env.step(env.action_space.sample())
    print('state:{} reward:{} done:{} prob:{}'.format(state, reward, done, prob))
    if done:
        break
env.close()

state:0 reward:0.0 done:False prob:{'prob': 0.3333333333333333}
state:0 reward:0.0 done:False prob:{'prob': 0.3333333333333333}
state:1 reward:0.0 done:False prob:{'prob': 0.3333333333333333}
state:5 reward:0.0 done:True prob:{'prob': 0.3333333333333333}


In [36]:
state_lookup = np.eye(env.observation_space.n)
action_lookup = np.eye(env.action_space.n)

In [86]:
lr = 0.001
memory_size = 2000
episodes = 1000
steps = 500
epsilon = 1
epsilon_min = 0.1
epsilon_decay = 0.999
gamma = 0.95
batch_size = 32

In [87]:
bot_flake = Sequential()
bot_flake.add(Dense(units=16, input_dim=16, activation='relu'))
bot_flake.add(Dense(units=4, activation='softmax'))
bot_flake.compile(loss='mse', optimizer=Adam(lr), metrics=['accuracy'])

In [88]:
memory = deque(maxlen = memory_size)

In [89]:
reward_list = []

env.reset()
state, reward, done, prob = env.step(env.action_space.sample())
for episode in range(episodes):
    env.reset()
    total_reward = 0
    for time in range(steps):
        if epsilon >= np.random.rand():
            action = env.action_space.sample()
        else:
            action = np.argmax(bot_flake.predict(np.reshape(state_lookup[state], [1,16])))
        next_state, reward, done, prob = env.step(np.argmax(action))
        memory.append((state, action, next_state, reward, done))
        state = next_state
        total_reward = total_reward + reward
        if done:
            print('episode:{} frames:{} total rewards:{} epsilon:{}'.format(episode + 1, time + 1, total_reward, epsilon))
            break
        if epsilon > epsilon_min:
            epsilon = epsilon * epsilon_decay
        
        minibatch = [memory[ii] for ii in np.random.choice(range(len(memory)),batch_size)]
        for state, action, next_state, reward, done in minibatch:
            Q_target = bot_flake.predict(np.reshape(state_lookup[state], [1,16]))
            Q_action = reward + gamma * np.argmax(bot_flake.predict(np.reshape(state_lookup[next_state], [1,16])))
            Q_target[0, action] = Q_action
            bot_flake.fit(np.reshape(state_lookup[state], [1,16]), Q_target, epochs=1, verbose=0)

episode:1 frames:10 total rewards:0.0 epsilon:0.9910359161258742
episode:2 frames:13 total rewards:0.0 epsilon:0.9792086759647052
episode:3 frames:30 total rewards:0.0 epsilon:0.9512056281970315
episode:4 frames:7 total rewards:0.0 epsilon:0.9455126435024219
episode:5 frames:18 total rewards:0.0 epsilon:0.9295668775782806
episode:6 frames:7 total rewards:0.0 epsilon:0.9240034012385749
episode:7 frames:15 total rewards:0.0 epsilon:0.9111511025165902
episode:8 frames:36 total rewards:0.0 epsilon:0.8797970327640969
episode:9 frames:57 total rewards:0.0 epsilon:0.831859218194368
episode:10 frames:17 total rewards:0.0 epsilon:0.818648829478636
episode:11 frames:12 total rewards:0.0 epsilon:0.8096885832327116
episode:12 frames:23 total rewards:0.0 epsilon:0.7920612314455105
episode:13 frames:19 total rewards:0.0 epsilon:0.7779246707428734
episode:14 frames:63 total rewards:0.0 epsilon:0.7311354045730207
episode:15 frames:5 total rewards:0.0 epsilon:0.7282152468433455
episode:16 frames:11 tot

KeyboardInterrupt: 

In [85]:
action

array([[ 0.3124204 ,  0.41373047,  0.1440673 ,  0.12978177]], dtype=float32)